## 2.Suggestions based on business attributes
In this part, we want to figure out which 5 business attributes are most important to ratings and how they influence ratings. We extracted imformation of business attributes from raw data, used tree-method to rank business attributes with importance score and did nonparametric tests to check if the top 5 business attributes are statistically related to ratings.
### 2.1 Business data processing
We first extracted all the business attributes from nested dictionary as features and examined the type of feature values. We transformed all the values with encoding string type like "u'average" into normal string type and all the values "None" with string type into true missing value type. Then we got the data with uniform string type and transformed them into categorical type to build model. We deleted those business attributes with more than 80 percent missing values. Finally, we got the 4272* 26 data, which means each restaurant has information on 26 bussiness attributes and there are 4272 restaurants in total.
### 2.2 Model fitting : rank of business attributes
We used average stars as y, data of business attributes as X. We chose GBDT(Gradient Boosting Decison Tree) to build our model because it can handle missing values automatically. We got the importance score of 26 business attributes based on this model and ranked them. We selected the top 5 business attributes for future work. They are RestaurantsTableService, OutdoorSeating, BusinessParking, NoiseLevel, Caters.
### 2.3 Hypothesis testing
We checked if the top 5 business attributes are really related to ratings through nonparametric tests Kruskal–Wallis H test and Wilcoxon rank-sum test because our data aren't distributed normally. Kruskal-Wallis H test is used for testing whether samples originate from the same distribution and it can compare more than two samples. The null hypothesis is that the samples are the same distribution, and the alternative hypothesis is that al least one distribution of one sample is different from the distribution of at least one other sample. Wilcoxon rank-sum test is extended from Kruskal-Wallis H test and is used for testing only 2 samples.[1]

1. For business attributes that have only 2 categories, we straightly applied Wilcoxon rank-sum test to see the exsitance of statistical difference between two categories.  If p-value < 0.05, the null hypothesis is rejected, we chose it as a candidate.
2. For business attributes that have more than 2 categories, we first applied Kruskal test to see if there is some difference exsiting in the different levels. If p-value < 0.05, the null hypothesis is rejected, then we did pairwise Wilcoxon test with Bonferroni correction to see the real difference among all the level pairs. Supposed that we have m tests and the overall alpha level is 0.05, we would test each individual hypothesis at a significance level of 0.05/m to control the familywise error rate.  

Through hypothesis tests, we found that the top 5 business attributes are statistically related to ratings. For NoiseLevel that has 4 levels, the ratings distribution of loud sample and very_loud sample are the same. 
### 2.4 Suggestions
We examined the meanings of these business attributes and found that the definition of Caters is ambiguous. So we retained the remaining 4 business attributes. For each business attribute, we examined the average star ratings of each level, sorted them in descending order and picked the level of highest average star ratings as the ideal level. Our final suggestions based on RestaurantsTableService, OutdoorSeating, BusinessParking, NoiseLevel (ordered by importance) are:
1. For restaurants that have table service, if they cancel the table service, they will increase the average rating by 0.18 stars.
2. For restaurants that have no ourdoor seating, if they add some outdoor seats, they will increase the average rating by 0.24 stars.
3. For restaurants that have no business parking place, if they provide some parking places for customers, they will increase the average star rating by 0.17 stars.
4. For restaurants that noise level is very loud and loud, if they provide a quiet environment for customers to enjoy food, they will increase the average star by 0.26 to 0.38 stars; for restaurants that noise level is average, they could make the environment quieter to increase the average star by 0.09 stars.

## 4. Regularized logistic regression prediction model
In this part, due to the limited memory of our computer, we randomly selected one million reviews as our training data, but it contained 87% business owners, so we thought that our data contained enough imformation. We used TFIDF score to construct design matrix and built a logistic regression model to predict star ratings on 1321274 reviews. 
### 4.1 Review data preprocessing
1. Transformed emoticons into English words. Eg. ":)" to "happy"
2. Expand abbreviation. Eg. "'m" to "am"
3. Convert text into lowercase.
4. Handle negation: add not_ to every word between negation and following punctuation.  
Eg. "The food did not taste good."  to "The food did not not_taste not_good."
5. Remove punctuation.
6. Lemmatize: reduce inflections or variant forms to base form
7. Remove unimportant words
8. Extract phrases. Eg. "french toast" to "french_toast"  

### 4.2 Feature extraction: TFIDF
We used TFIDF score to construct design matrix. TFIDF is used to reflect the importance of a word in a text. Higher TFIDF score indicates higher importance. Supposed that t indicates a term and d indicate a document. Then the definition of TFIDF is:

$tf(t,d) =$number of times term t appears in document d  
$idf(t) =$number of documents in collection devided by number of documents that contain t  
$TFIDF score = tf(t,d) * idf(t)$  
Finally, we got the design matrix with $1000000* 628798$ dimension and we used this to train our logistic regression model.
### 4.3 Model fitting
We splited design matrix into train set and dev set. We used train set to construct regularized logistic regression model and dev set to evaluate the model perfomance. We used SAGA, a fast stochastic optimization method usually used in large amount of data to optimize the cost function. To avoid overfiiting, we also added L2 penalty to our cost function. The smallest RMSE 0.6572 on dev set is the model that uses 2.2 as inverse of regularization strength. So we picked this as our final model parameters and used all the design matrix to train final prediction model.    
The RMSE on test set is 0.65552, which shows that our prediction model performs well. Also, from the coefficients in this model, we found that for 1 star the word with largest coefficient is 'give_zero' and for 5 star the word with largest coefficient is 'highly_recommend'. It also gives some insights on performance of our model because intuitively 'give_zero' is a negative word and 'highly_recommend' is a positive word.